In [ ]:
%%capture
!pip install streamlit pyngrok PyPDF2 langchain transformers huggingface_hub sentence-transformers


In [ ]:
import os
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from transformers import pipeline
from langchain.llms import HuggingFaceHub


# Çalışan Streamlit uygulamasını durdur
ngrok.kill()

# Çalıştırılacak portu ayarla
port = 8501
public_url = ngrok.connect(port).public_url
print(f"📢 Public URL: {public_url}")

# Huggingface için token yükleme
from huggingface_hub import login

# Hugging Face Token'ını Buraya Gir
HUGGINGFACE_TOKEN = "login(token=HUGGINGFACE_TOKEN)

# PDF Dosyasından Metin Çıkarma Fonksiyonu
def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


# Metni Parçalara Bölme (Chunking)
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks


# Vektör Veritabanı Oluşturma
def get_vectorstore(text_chunks):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore


# Konuşma Zinciri Oluşturma
def get_conversation_chain(vectorstore):
    # Meta-Llama modelini başlatma
    llm = HuggingFaceHub(
        repo_id="meta-llama/Llama-3.2-3B-Instruct",
        model_kwargs={"temperature": 0.5, "max_length": 512}
    )

    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True
    )

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain


# Kullanıcı Sorularını İşleme Fonksiyonu
def handle_userinput(user_question):
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    for i, message in enumerate(st.session_state.chat_history):
        if i % 2 == 0:
            st.markdown(f"**Kullanıcı:** {message.content}")
        else:
            st.markdown(f"**Bot:** {message.content}")


# Ana Uygulama Fonksiyonu
def main():
    # Çevre değişkenlerini yükle
    load_dotenv()

    # Sayfa başlığı ve favicon
    st.set_page_config(page_title="PDF Chatbot", page_icon="📄")

    st.title("📄 Birden Fazla PDF ile Sohbet Edin!")
    st.write("PDF dosyalarınızı yükleyin ve akıllı asistan ile sohbet edin.")

    # Uygulama Durumu Başlatma
    if "conversation" not in st.session_state:
        st.session_state.conversation = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = None

    # Kullanıcıdan Soru Alma
    user_question = st.text_input("Belgelerinizle ilgili bir soru sorun:")
    if user_question:
        handle_userinput(user_question)

    with st.sidebar:
        st.subheader("📂 Dosyalarınızı Yükleyin")
        pdf_docs = st.file_uploader(
            "PDF dosyalarınızı buraya yükleyin ve ardından 'İşle' butonuna tıklayın.",
            accept_multiple_files=True
        )

        if st.button("📌 İşle"):
            with st.spinner("🚀 İşleniyor..."):
                # PDF'ten metin çıkarma
                raw_text = get_pdf_text(pdf_docs)

                # Metni parçalara bölme (chunking)
                text_chunks = get_text_chunks(raw_text)

                # Vektör veritabanı oluşturma
                vectorstore = get_vectorstore(text_chunks)

                # Konuşma zinciri başlatma
                st.session_state.conversation = get_conversation_chain(vectorstore)

                st.success("✅ İşlem tamamlandı! Şimdi sorularınızı sorabilirsiniz.")

# Streamlit'i çalıştırma işlevi
def start_streamlit():
    os.system("streamlit run app.py")

# Uygulamayı arka planda çalıştır
thread = threading.Thread(target=start_streamlit)
thread.start()

# Uygulamayı başlat
if __name__ == '__main__':
    main()
